# Collect data

In [1]:
import pandas as pd

df = pd.read_csv('data/base.csv', sep=';')

/tmp/ipykernel_10738/4069637141.py:3: DtypeWarning: Columns (4,11,18,19,20,21,22,27,28,31,32,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/base.csv', sep=';')


# Filter data

## Remove missing data

### Inspect missing values

In [2]:
df.isna().sum()

Unnamed: 0                            0
sintomas                            334
racaCor                           21583
outrosSintomas                  3292526
outrasCondicoes                 6240233
cbo                             6081056
condicoes                       5733568
sexo                                 41
estado                               61
estadoIBGE                      1234517
municipio                            67
municipioIBGE                   1248796
origem                              369
estadoNotificacao                     0
municipioNotificacao              27355
municipioNotificacaoIBGE        2558498
evolucaoCaso                    1147649
classificacaoFinal              1228707
codigoLaboratorioSegundaDose    4167656
lotePrimeiraDose                3834313
loteSegundaDose                 4167812
codigoDosesVacina               3621993
estadoNotificacaoIBGE           2531598
totalTestesRealizados                 4
dataNotificacao                   29660


### Drop high missing-rate columns

In [3]:
threshold = 0.9 * len(df)
df = df.loc[:, df.isna().sum() < threshold]

### Drop missing-value records

In [4]:
df.dropna(inplace=True)

## Remove future-bound columns

### Inspect columns

In [5]:
df.columns

Index(['Unnamed: 0', 'sintomas', 'racaCor', 'outrosSintomas', 'condicoes',
       'sexo', 'estado', 'estadoIBGE', 'municipio', 'municipioIBGE', 'origem',
       'estadoNotificacao', 'municipioNotificacao', 'municipioNotificacaoIBGE',
       'evolucaoCaso', 'classificacaoFinal', 'codigoLaboratorioSegundaDose',
       'lotePrimeiraDose', 'loteSegundaDose', 'codigoDosesVacina',
       'estadoNotificacaoIBGE', 'totalTestesRealizados', 'dataNotificacao',
       'dataInicioSintomas', 'dataEncerramento', 'dataPrimeiraDose',
       'dataSegundaDose', 'codigoResultadoTeste1', 'dataColetaTeste1', 'idade',
       'faixa_etaria'],
      dtype='object')

### Drop columns

In [6]:
df.drop(df.columns[[14, 24]], axis=1, inplace=True)

## Remove useless columns

### Drop columns

In [7]:
df.drop(df.columns[[0, 10]], axis=1, inplace=True)

## Remove redundant data

### Inspect columns

In [8]:
df.columns

Index(['sintomas', 'racaCor', 'outrosSintomas', 'condicoes', 'sexo', 'estado',
       'estadoIBGE', 'municipio', 'municipioIBGE', 'estadoNotificacao',
       'municipioNotificacao', 'municipioNotificacaoIBGE',
       'classificacaoFinal', 'codigoLaboratorioSegundaDose',
       'lotePrimeiraDose', 'loteSegundaDose', 'codigoDosesVacina',
       'estadoNotificacaoIBGE', 'totalTestesRealizados', 'dataNotificacao',
       'dataInicioSintomas', 'dataPrimeiraDose', 'dataSegundaDose',
       'codigoResultadoTeste1', 'dataColetaTeste1', 'idade', 'faixa_etaria'],
      dtype='object')

### Drop columns

In [9]:
df.drop(df.columns[[5, 7, 9, 10, 26]], axis=1, inplace=True)

## Remove biasable columns

### Inspect data

In [10]:
df.columns

Index(['sintomas', 'racaCor', 'outrosSintomas', 'condicoes', 'sexo',
       'estadoIBGE', 'municipioIBGE', 'municipioNotificacaoIBGE',
       'classificacaoFinal', 'codigoLaboratorioSegundaDose',
       'lotePrimeiraDose', 'loteSegundaDose', 'codigoDosesVacina',
       'estadoNotificacaoIBGE', 'totalTestesRealizados', 'dataNotificacao',
       'dataInicioSintomas', 'dataPrimeiraDose', 'dataSegundaDose',
       'codigoResultadoTeste1', 'dataColetaTeste1', 'idade'],
      dtype='object')

### Drop columns

In [11]:
df.drop(df.columns[[19, 20]], axis=1, inplace=True)

## Select the two main labels

### Inspect labels

In [12]:
df['classificacaoFinal'].unique()

array(['Descartado', 'Confirmado Laboratorial',
       'Confirmado Clínico-Epidemiológico',
       'Síndrome Gripal Não Especificada', 'Confirmado Clínico-Imagem',
       'Confirmado por Critério Clínico'], dtype=object)

### Remove records where the label isn't "Confirmado Laboratorial" or "Descartado"

In [13]:
df = df[df['classificacaoFinal'].isin(['Confirmado Laboratorial', 'Descartado'])]

### Note
Before removing unwanted labels, there where 50.060. After the removal, 50.048 have left.

Would this suggest the dataset is, somehow, biased (providing more consistant data) to these two classes?

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50048 entries, 876563 to 6379912
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   sintomas                      50048 non-null  object 
 1   racaCor                       50048 non-null  object 
 2   outrosSintomas                50048 non-null  object 
 3   condicoes                     50048 non-null  object 
 4   sexo                          50048 non-null  object 
 5   estadoIBGE                    50048 non-null  object 
 6   municipioIBGE                 50048 non-null  object 
 7   municipioNotificacaoIBGE      50048 non-null  float64
 8   classificacaoFinal            50048 non-null  object 
 9   codigoLaboratorioSegundaDose  50048 non-null  object 
 10  lotePrimeiraDose              50048 non-null  object 
 11  loteSegundaDose               50048 non-null  object 
 12  codigoDosesVacina             50048 non-null  object 
 13 

## Equalize labels distribution

### Inspect labels distribution

In [15]:
df['classificacaoFinal'].value_counts()

classificacaoFinal
Descartado                 36079
Confirmado Laboratorial    13969
Name: count, dtype: int64

### Undersample labels

In [16]:
confirmed_df = df[df["classificacaoFinal"] == "Confirmado Laboratorial"]
discarded_df = df[df["classificacaoFinal"] == "Descartado"]

balanced_df = pd.concat(
    [
        confirmed_df.sample(n=10_000, random_state=42),
        discarded_df.sample(n=10_000, random_state=42),
    ]
)

# Shuffle the dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(
    drop=True
)

# Arrange data

## Arrange symptoms

### Inspect data

In [17]:
balanced_df.head()

,sintomas,racaCor,outrosSintomas,condicoes,sexo,estadoIBGE,municipioIBGE,municipioNotificacaoIBGE,classificacaoFinal,codigoLaboratorioSegundaDose,lotePrimeiraDose,loteSegundaDose,codigoDosesVacina,estadoNotificacaoIBGE,totalTestesRealizados,dataNotificacao,dataInicioSintomas,dataPrimeiraDose,dataSegundaDose,idade
0,Febre,Branca,"Disturbios olfativos, outros",Imunossupressão,Masculino,PE,2611606.0,2611606.0,Descartado,SINOVAC/BUTANTAN,210344,210453,"1,2,3",PE,1.0,2024-02-16,2024-02-07,2021-08-12,2021-09-26,36.0
1,Tosse,Parda,"Coriza, dor de garganta, outros",Diabetes,Feminino,PE,2604106.0,2604106.0,Confirmado Laboratorial,ASTRAZENECA,214VCD054Z,210197,"2,1",PE,1.0,2022-06-22,2022-06-20,2021-05-14,2021-08-20,52.0
2,Febre,Parda,"Coriza, tosse, dor de garganta, outros","Diabetes, Imunossupressão",Feminino,PE,2615300.0,2615300.0,Confirmado Laboratorial,SINOVAC/BUTANTAN,210080,210114,"1,2",PE,1.0,2022-02-04,2022-02-04,2021-03-24,2021-04-14,30.0
3,Tosse,Parda,"Coriza, dor de cabeca, dor de garganta, outros",Doenças respiratórias crônicas descompensadas,Feminino,PE,2611606.0,2611606.0,Confirmado Laboratorial,ASTRAZENECA/FIOCRUZ,214VCD107Z,217VCD234Z,"1,2",PE,1.0,2022-02-03,2022-01-28,2021-06-06,2021-08-30,37.0
4,Tosse,Branca,"Dor de cabeca, dor de garganta",Outros,Masculino,PE,2611606.0,2613701.0,Descartado,PFIZER,EW0199,FD7209,"1,2",PE,1.0,2022-04-12,2022-04-09,2021-05-29,2021-08-27,42.0


### Merge symptoms columns

In [18]:
balanced_df['todosSintomas'] = balanced_df['sintomas'].str.upper() + ', ' + balanced_df['outrosSintomas'].str.upper()

balanced_df.drop(['sintomas', 'outrosSintomas'], axis=1, inplace=True)

## Encode string-based categorical data

### Setup LabelEncoder

In [19]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

### Encode columns

In [20]:
balanced_df['racaCor'] = label_encoder.fit_transform(balanced_df['racaCor'])
balanced_df['sexo'] = label_encoder.fit_transform(balanced_df['sexo'])
balanced_df['estadoIBGE'] = label_encoder.fit_transform(balanced_df['estadoIBGE'])
balanced_df['classificacaoFinal'] = label_encoder.fit_transform(balanced_df['classificacaoFinal'])
balanced_df['codigoLaboratorioSegundaDose'] = label_encoder.fit_transform(balanced_df['codigoLaboratorioSegundaDose'])
balanced_df['lotePrimeiraDose'] = label_encoder.fit_transform(balanced_df['lotePrimeiraDose'])
balanced_df['loteSegundaDose'] = label_encoder.fit_transform(balanced_df['loteSegundaDose'])
balanced_df['estadoNotificacaoIBGE'] = label_encoder.fit_transform(balanced_df['estadoNotificacaoIBGE'])

## Parse integers and datetimes

### Inspect data

In [21]:
balanced_df.head()

,racaCor,condicoes,sexo,estadoIBGE,municipioIBGE,municipioNotificacaoIBGE,classificacaoFinal,codigoLaboratorioSegundaDose,lotePrimeiraDose,loteSegundaDose,codigoDosesVacina,estadoNotificacaoIBGE,totalTestesRealizados,dataNotificacao,dataInicioSintomas,dataPrimeiraDose,dataSegundaDose,idade,todosSintomas
0,1,Imunossupressão,1,4,2611606.0,2611606.0,1,21,92,113,"1,2,3",0,1.0,2024-02-16,2024-02-07,2021-08-12,2021-09-26,36.0,"FEBRE, DISTURBIOS OLFATIVOS, OUTROS"
1,4,Diabetes,0,4,2604106.0,2604106.0,0,3,157,71,"2,1",0,1.0,2022-06-22,2022-06-20,2021-05-14,2021-08-20,52.0,"TOSSE, CORIZA, DOR DE GARGANTA, OUTROS"
2,4,"Diabetes, Imunossupressão",0,4,2615300.0,2615300.0,0,21,47,42,"1,2",0,1.0,2022-02-04,2022-02-04,2021-03-24,2021-04-14,30.0,"FEBRE, CORIZA, TOSSE, DOR DE GARGANTA, OUTROS"
3,4,Doenças respiratórias crônicas descompensadas,0,4,2611606.0,2611606.0,0,5,182,201,"1,2",0,1.0,2022-02-03,2022-01-28,2021-06-06,2021-08-30,37.0,"TOSSE, CORIZA, DOR DE CABECA, DOR DE GARGANTA,..."
4,1,Outros,1,4,2611606.0,2613701.0,1,13,262,271,"1,2",0,1.0,2022-04-12,2022-04-09,2021-05-29,2021-08-27,42.0,"TOSSE, DOR DE CABECA, DOR DE GARGANTA"


### Parse numeric columns

In [22]:
balanced_df["municipioIBGE"] = pd.to_numeric(balanced_df["municipioIBGE"]).astype(int)
balanced_df['municipioNotificacaoIBGE'] = pd.to_numeric(balanced_df['municipioNotificacaoIBGE']).astype(int)
balanced_df['totalTestesRealizados'] = pd.to_numeric(balanced_df['totalTestesRealizados']).astype(int)
balanced_df['idade'] = pd.to_numeric(balanced_df['idade']).astype(int)

### Parse datetime columns

In [23]:
balanced_df['dataNotificacao'] = pd.to_datetime(balanced_df['dataNotificacao'])
balanced_df['dataInicioSintomas'] = pd.to_datetime(balanced_df['dataInicioSintomas'])
balanced_df['dataPrimeiraDose'] = pd.to_datetime(balanced_df['dataPrimeiraDose'])
balanced_df['dataSegundaDose'] = pd.to_datetime(balanced_df['dataSegundaDose'])

balanced_df['dataNotificacao'] = balanced_df['dataNotificacao'].apply(lambda x: x.timestamp())
balanced_df['dataInicioSintomas'] = balanced_df['dataInicioSintomas'].apply(lambda x: x.timestamp())
balanced_df['dataPrimeiraDose'] = balanced_df['dataPrimeiraDose'].apply(lambda x: x.timestamp())
balanced_df['dataSegundaDose'] = balanced_df['dataSegundaDose'].apply(lambda x: x.timestamp())

### Normalize timestamp scales

In [24]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

balanced_df["dataNotificacao"] = scaler.fit_transform(
    balanced_df["dataNotificacao"].values.reshape(-1, 1)
)
balanced_df["dataInicioSintomas"] = scaler.fit_transform(
    balanced_df["dataInicioSintomas"].values.reshape(-1, 1)
)
balanced_df["dataPrimeiraDose"] = scaler.fit_transform(
    balanced_df["dataPrimeiraDose"].values.reshape(-1, 1)
)
balanced_df["dataSegundaDose"] = scaler.fit_transform(
    balanced_df["dataSegundaDose"].values.reshape(-1, 1)
)

## Binarize multi-label columns

The OneHotEncoder from sklearn.preprocessing is designed to work with categorical data where each entry is a single category. It does not handle lists of categories directly. When you pass a list of categories to OneHotEncoder, it treats the entire list as a single category, which leads to errors.

On the other hand, MultiLabelBinarizer is specifically designed to handle lists of categories. It can take a list of categories for each entry and one-hot encode them correctly.

### Inspect data

In [25]:
balanced_df.head()

,racaCor,condicoes,sexo,estadoIBGE,municipioIBGE,municipioNotificacaoIBGE,classificacaoFinal,codigoLaboratorioSegundaDose,lotePrimeiraDose,loteSegundaDose,codigoDosesVacina,estadoNotificacaoIBGE,totalTestesRealizados,dataNotificacao,dataInicioSintomas,dataPrimeiraDose,dataSegundaDose,idade,todosSintomas
0,1,Imunossupressão,1,4,2611606,2611606,1,21,92,113,"1,2,3",0,1,0.859375,0.878412,0.239486,0.209748,36,"FEBRE, DISTURBIOS OLFATIVOS, OUTROS"
1,4,Diabetes,0,4,2604106,2604106,0,3,157,71,"2,1",0,1,0.409970,0.508065,0.134346,0.177546,52,"TOSSE, CORIZA, DOR DE GARGANTA, OUTROS"
2,4,"Diabetes, Imunossupressão",0,4,2615300,2615300,0,21,47,42,"1,2",0,1,0.307292,0.423697,0.074766,0.066144,30,"FEBRE, CORIZA, TOSSE, DOR DE GARGANTA, OUTROS"
3,4,Doenças respiratórias crônicas descompensadas,0,4,2611606,2611606,0,5,182,201,"1,2",0,1,0.306548,0.419355,0.161215,0.186249,37,"TOSSE, CORIZA, DOR DE CABECA, DOR DE GARGANTA,..."
4,1,Outros,1,4,2611606,2613701,1,13,262,271,"1,2",0,1,0.357143,0.463400,0.151869,0.183638,42,"TOSSE, DOR DE CABECA, DOR DE GARGANTA"


### Turn comma-separated columns into lists

In [26]:
import re

balanced_df["codigoDosesVacina"] = balanced_df["codigoDosesVacina"].str.split(",")
balanced_df["todosSintomas"] = (
    balanced_df["todosSintomas"]
    .str.split(",")
    .apply(lambda x: list(set(item.strip().upper() for item in x)))
)
balanced_df["condicoes"] = (
    balanced_df["condicoes"]
    .apply(lambda x: re.sub(r'\(.*?\)', '', x))
    .str
    .split(",")
    .apply(lambda x: list(set(item.strip().upper() for item in x)))
)

### Setup MultiLabelBinarizer

In [27]:
from sklearn.preprocessing import MultiLabelBinarizer

multi_label_binarizer = MultiLabelBinarizer()

### Encode columns

In [28]:
from utils import dose_ordinals


def to_camel_case(s: str) -> str:
    parts = s.lower().split()
    camel_case_string = parts[0] + "".join(word.capitalize() for word in parts[1:])
    return camel_case_string


encoded_doses = multi_label_binarizer.fit_transform(balanced_df["codigoDosesVacina"])
encoded_doses_df = pd.DataFrame(
    encoded_doses,
    columns=[f"{dose_ordinals[int(n)]}Dose" for n in multi_label_binarizer.classes_],
)

encoded_symptoms = multi_label_binarizer.fit_transform(balanced_df["todosSintomas"])
encoded_symptoms_df = pd.DataFrame(
    encoded_symptoms,
    columns=[to_camel_case(sintoma) for sintoma in multi_label_binarizer.classes_],
)

encoded_conditions = multi_label_binarizer.fit_transform(balanced_df["condicoes"])
encoded_conditions_df = pd.DataFrame(
    encoded_conditions,
    columns=[to_camel_case(condicao) for condicao in multi_label_binarizer.classes_],
)

balanced_df = pd.concat(
    [balanced_df, encoded_doses_df, encoded_symptoms_df, encoded_conditions_df], axis=1
).drop(["codigoDosesVacina", "todosSintomas", "condicoes"], axis=1)

# Split data

## Inspect data

In [29]:
balanced_df.head()

,racaCor,sexo,estadoIBGE,municipioIBGE,municipioNotificacaoIBGE,classificacaoFinal,codigoLaboratorioSegundaDose,lotePrimeiraDose,loteSegundaDose,estadoNotificacaoIBGE,...,diabetes,doençasCardíacasCrônicas,doençasRenaisCrônicasEmEstágioAvançado,doençasRespiratóriasCrônicasDescompensadas,gestante,imunossupressão,obesidade,outros,portadorDeDoençasCromossômicasOuEstadoDeFragilidadeImunológica,puérpera
0,1,1,4,2611606,2611606,1,21,92,113,0,...,0,0,0,0,0,1,0,0,0,0
1,4,0,4,2604106,2604106,0,3,157,71,0,...,1,0,0,0,0,0,0,0,0,0
2,4,0,4,2615300,2615300,0,21,47,42,0,...,1,0,0,0,0,1,0,0,0,0
3,4,0,4,2611606,2611606,0,5,182,201,0,...,0,0,0,1,0,0,0,0,0,0
4,1,1,4,2611606,2613701,1,13,262,271,0,...,0,0,0,0,0,0,0,1,0,0


## Split by prediction label

In [30]:
from sklearn.model_selection import train_test_split

X = balanced_df.drop("classificacaoFinal", axis=1)
y = balanced_df["classificacaoFinal"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Distribute data

In [31]:
pd.concat([X_train, y_train], axis=1).to_csv("data/symptoms-with-conditions-train.csv", index=False)
pd.concat([X_test, y_test], axis=1).to_csv("data/symptoms-with-conditions-test.csv", index=False)